In [ ]:
from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
from pathlib import Path
import pandas as pd
import pickle
import seaborn as sns
import shutil
import time

from graphs.mixture_fit import best_fit_mixture
from models.gaussian_mixture import remove_outliers, gaussian_mixture
from preprocessing.exoplanets_gaia_crossmatch import gaia_exoplanets_cross, transform_to_cart
from preprocessing.download_gaia import GaiaDataset
from preprocessing.calc_density import get_densities

## Create folder structure

In [ ]:
crossmatch_dir = "data/crossmatch/dr3"
densities_dir = "data/densities/dr3"
classification_dir = "data/classification/dr3"
datasets_dir = "data/initial_datasets"

# 1. Login

#### First let's login to Gaia archive. We can use the following formats:
#### Gaia.login_gui() - will display window to provide a username and password
#### Gaia.login() - a prompt will ask the user for name and password
#### Gaia.login(user=[username], password=[password]) - just type the username and password in the given fields
#### Gaia.login(credentials_file=[filename] - A path to the file that contains username and password. The username and password must be in different lines!

In [ ]:
Gaia.login()

# 2. Prepare Exoplanets data for crossmatching query.
### Remove any targets that do not have GAIA ID. Extract numerical value of GAIA ID and save it as integer. Remove duplicates so that we end up with a single instance of each host star.

In [ ]:
df = pd.read_csv("data/initial_datasets/PSCompPars_2022.11.10_06.54.37.csv", skiprows=9)
df.dropna(subset=["gaia_id"], inplace=True)
df["source_id"] = df["gaia_id"].str.rsplit(" ", n=1, expand=True)[1].dropna().astype("int64")
df["Host"] = df["hostname"].str.replace(" ", "")
df.drop_duplicates(subset=["Host"], inplace=True)
df.drop(["gaia_id", "hostname"], axis=1, inplace=True)
df.to_csv("data/initial_datasets/NASA_ex_cleaned.csv", index=False)

# 3. Upload table with Gaia ID's from NASA Exoplanets Archive
### Can be done on the Gaia archive website using GUI. Cannot upload the same table twice unless we change the name as they are unique.

In [ ]:
#job = Gaia.delete_user_table("exoplanets")
Gaia.upload_table(upload_resource="data/initial_datasets/NASA_ex_cleaned.csv", table_name="exoplanets", format="CSV")

# 4. Submit crossmatch query

##### Provide gaia table name to query as well as your username that will be used to load the uploaded table.
##### Set a name for the file to which queried data will be saved.

In [ ]:
table = "gaiadr3.dr2_neighbourhood"
username = "mmotylin"
filename = "exoplanets_query.csv"

In [ ]:
query = f"""
SELECT exoplanets.*, both_names.dr2_source_id, both_names.dr3_source_id
FROM user_{username}.exoplanets AS exoplanets
JOIN {table} as both_names 
    ON both_names.dr2_source_id = exoplanets.source_id
WHERE ABS(magnitude_difference) < 0.1
"""

In [ ]:
Gaia.launch_job_async(query).get_results().to_pandas().to_csv(f"data/initial_datasets/{filename}", index=False)

# 5. Clean up after query.

In [ ]:
df1 = pd.read_csv(f"data/initial_datasets/{filename}")
df1["source_id"] = df1["dr3_source_id"]
df1 = df1[["pl_name", "source_id", "host"]]
df1.to_csv("data/initial_datasets/exoplanets.csv", index=False)

# 6. Crossmatch NASA Exoplanet dataset with Gaia dataset

In [ ]:
def exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="6d", table_name="gaiadr3", columns=["pl_name", "hostname", "gaia_id"], save_gaia_id=True, save_spherical=True):
    """
    :param: transform_type: Type of coordinates transformation to perform on the data (6d, 5d_drop_vx, 5d_drop_vy or 5d_drop_vz).
    :param: table_name: Name of the Gaia dataset to use.
    :param: save_spherical: Save spherical values to a CSV file. When looping it is adviced to apply only once to save time. 
    
    :return: Density values for 1065 exoplanets and their neighbours, Winter-Gaia-NASA exoplanet archive crossmatch
    table containing 6D coordinates only and Winter-Gaia-NASA exoplanet archive crossmatch table with data from all 3
    sources combined.
    """

    # Cross match datasets and generate new ones.
    gaia = gaia_exoplanets_cross(f"{table_name}.csv", crossmatch_dir, columns, save_gaia_id=True, return_data=True, save_spherical=save_spherical)
    transform_to_cart(gaia, table_name, crossmatch_dir, setting=transform_type)

In [ ]:
columns = ["pl_name", "host", "gaia_id"]

In [ ]:
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="6d", columns=columns, save_gaia_id=True, save_spherical=True)
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="5d_drop_rv", columns=columns, save_gaia_id=False, save_spherical=False)

# 7. Calculate phase space density for neighbours of exoplanet hosts

In [ ]:
def calculate_densities(star_labels_filename, dataset_filename, crossmatch_dir, densities_dir, exoplanets_only=True, start=0, stop=100000, step=1, run_on_gpu=False):
    """
    Calculate phase space density for given set of stars.
    
    :param: star_labels_filename: Name of the file containing star labels.
    :param: dataset_filename: Name of the file containing coordinates of the stars.
    :param: exoplanets_only: Compute density only for a list of exoplanets (~1000).
    :param: n_stars: Numeber of stars to calculate density for.
    :param: run_on_gpu: Use GPU accelerated pipeline.
    """
    
    labels_file = pd.read_csv(os.path.join(crossmatch_dir, star_labels_filename), dtype={"source_id": str, "host": str})
    gaia = pd.read_csv(os.path.join(crossmatch_dir, dataset_filename))
                
    if gaia.shape[1] == 6:
        name = dataset_filename.split("_")[0] + f"_{dataset_filename[-6:-4]}"
    else:
        name = dataset_filename.split("_")[0] + f"_{dataset_filename[-14:-4]}"

    if exoplanets_only:
        labels = labels_file["host"].dropna()
        start = 0
        stop = labels.shape[0]
        step = 1
        name = name + "_only-" + dataset_filename.split("_")[1] + "s"
    else:
        labels = labels_file["source_id"]
        name = name + f"_{str(start)}_{str(stop)}"


    densities, dropped = get_densities(labels.to_numpy(), gaia.to_numpy(), start=start, stop=stop, step=step, run_on_gpu=run_on_gpu)
    
    with open(f"{densities_dir}/densities_{name}.data", "wb") as f:
        pickle.dump(densities, f)
    with open(f"{densities_dir}/dropped_densities_{name}.data", "wb") as f:
        pickle.dump(dropped, f)

In [ ]:
for fname in os.listdir(crossmatch_dir):
    if "cartesian" in fname:
        print(fname)
        start = time.perf_counter()
        calculate_densities("gaiadr3_star_labels.csv", fname, crossmatch_dir, densities_dir, exoplanets_only=True, run_on_gpu=True)
        end = time.perf_counter()
        print(f"{fname} completed in: {str(end-start)}")


# 8. Fit gaussian mixture model to predict if target star belongs to overdensity or underdensity group. Return scores and attributes of the model.

In [ ]:
def labels(row):
    if row["gm_p_low"] >= 0.84:
        return "0"
    elif row["gm_p_high"] >= 0.84:
        return "2"
    else:
        return "1"

In [ ]:
import numpy as np
import math
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from scipy.stats import kstest
from scipy import stats
from math import exp

In [ ]:
def fit_gaussian_mixture(file_name, classification_dir, densities_dir, show_graph=False, save_graph=False):
    if "5d" in file_name:
        data_type = "5d"
    else:
        data_type = "6d"
    sigma = 2
    fig_dir=None
    file_name = file_name.split(".")[0]
    if save_graph:  
        if os.path.isdir(f"figures/{file_name}"):
            shutil.rmtree(f"figures/{file_name}")
        fig_dir = file_name
        
        os.mkdir(f"figures/{fig_dir}")
    
    with open(f"{densities_dir}/{file_name}.data", "rb") as f:
        densities = pickle.load(f)
    results = []
        
    for i in densities:
        # Compute log10 of the host density and expand dimensions for further use
        target = np.expand_dims(np.log10(i[1]), axis=0).T

        # Remove outliers outside sigma
        data = remove_outliers(i[4], sigma=sigma)
        
        mean = np.mean(data)
        std = np.std(data)
        st, pnull = kstest(data.reshape(-1, ), 'norm', args=(mean, std))
        
        if pnull < 0.05:
            # Apply gaussian mixture model to the data
            model, scores = gaussian_mixture(data, [target], components=2, scores_only=False)

            # Create list consisting of star name and its density for graph drawing
            scores.insert(0, pnull)
            scores.insert(0, target[0])
            scores.insert(0, np.log10(i[4].min()))
            scores.insert(0, np.log10(i[4].max()))
            scores.insert(0, np.log10(i[4].std()))
            scores.insert(0, np.log10(i[4].mean()))
            scores.insert(0, i[3])
            scores.insert(0, i[2])
            scores.insert(0, i[0])
            results.append(scores)

            # Draw best fit mixture
            if type(i[0]) != str:
                host = [f"{i[0]:.0f}", target]
            else:
                host = [i[0], target]
                
            if show_graph or save_graph:
                best_fit_mixture(model, data, host, results[-1][9], pnull, fig_dir, show_graph, save_graph)
    df = pd.DataFrame(results, columns=["Host", "n_40pc_stars", "n_80pc_stars", "densities_mean", "densities_std", 
                                        "densities_max", "densities_min", "target_density", "gm_p_null", "gm_p_low", 
                                        "gm_p_high", "gm_mean_low", "gm_mean_high", "gm_cov_low", "gm_cov_high", 
                                        "gm_aic", "gm_bic"])
    df["class"] = df.apply(lambda row: labels(row), axis=1)
    
    df.to_csv(f"{classification_dir}/features_{file_name}.csv")

In [ ]:
features = os.listdir(classification_dir)
for fname in os.listdir(densities_dir):
    for i in features:
        if fname.split(".")[0] in i:
            continue
    if "dropped" in fname:
        continue
    fit_gaussian_mixture(fname, classification_dir, densities_dir, show_graph=False, save_graph=True)

# Old ks tests

# displaying densities against normal distribution

In [ ]:
def fit_gaussian_mixture(file_name, classification_dir, densities_dir, show_graph=False, save_graph=False):
    if "5d" in file_name:
        data_type = "5d"
    else:
        data_type = "6d"
    sigma = 2
    fig_dir=None
    file_name = file_name.split(".")[0]
    if save_graph:  
        if os.path.isdir(f"figures/{file_name}"):
            shutil.rmtree(f"figures/{file_name}")
        fig_dir = file_name
    
        os.mkdir(f"figures/{fig_dir}")
    
    with open(f"{densities_dir}/{file_name}.data", "rb") as f:
        densities = pickle.load(f)
    results = []
    l = 0
    k = 0
    arr = []
    for i in densities:
        # Compute log10 of the host density and expand dimensions for further use
        target = np.expand_dims(np.log10(i[1]), axis=0).T

        # Remove outliers outside sigma
        data = remove_outliers(i[4], sigma=sigma)
        
        mean = np.mean(data)
        std = np.std(data)
        
        x = np.random.normal(mean, std, data.reshape(-1, ).shape[0])
        
        st, p_null = kstest(data.reshape(-1, ), 'norm', args=(mean, std))
        
        if p_null > 0.05:
            print(test_stat)
            fig = plt.figure(figsize=(50, 5), facecolor="w")
            fig.subplots_adjust(left=0.12, right=0.97, bottom=0.21, top=0.9, wspace=0.5)
            ax = fig.add_subplot(131)
            ax.hist(data.reshape(-1, ), 30, density=True, histtype="stepfilled", alpha=0.5)
            ax.hist(x, 30, density=True, histtype="stepfilled", alpha=0.5, color="g")
            plt.show()
            plt.close()

### with probability density function

In [ ]:
def fit_gaussian_mixture(file_name, classification_dir, densities_dir, show_graph=False, save_graph=False):

    sigma = 2
    fig_dir=None
    file_name = file_name.split(".")[0]
    if save_graph:  
        if os.path.isdir(f"figures/{file_name}"):
            shutil.rmtree(f"figures/{file_name}")
        fig_dir = file_name
    
        os.mkdir(f"figures/{fig_dir}")
    
    with open(f"{densities_dir}/{file_name}.data", "rb") as f:
        densities = pickle.load(f)
    results = []
    l = 0
    k = 0
    arr = []

    for i in densities:
        # Compute log10 of the host density and expand dimensions for further use
        target = np.expand_dims(np.log10(i[1]), axis=0).T

        # Remove outliers outside sigma
        data = remove_outliers(i[4], sigma=sigma)
        
        model = GaussianMixture(n_components=1, max_iter=1000).fit(data)
        print(model.predict_proba(np.array([target])))
        print("means", model.means_, "covariances", model.covariances_)
        x = np.linspace(math.floor(data.min()), math.ceil(data.max()), data.shape[0])
        logprob = model.score_samples(x.reshape(-1, 1))
        pdf = np.exp(logprob)
        
        print("pdf range", pdf.min(), pdf.max())
        
        
        
        cdf = norm.cdf(data)
        print("cdf", cdf)
        
        params = model.get_params()
        print("params", params)
        
        #pval = 
        print("Densities range: ", data.min(), data.max())
        print("Densities: ", data, "linspace", x, "\n Logprob: ", logprob, "\n PDF: ", pdf)
        #test_stat = kstest(np.array([target]).reshape(-1, ), pdf)
        test_stat = ks_2samp(cdf.reshape(-1, ), data.reshape(-1, ))
        print(test_stat)
        #test_stat0 = kstest(np.exp(np.array([target])).reshape(-1, ), pdf)
        #test_stat1 = kstest(np.array([target]).reshape(-1, ), data.reshape(-1, ))
        fig = plt.figure(figsize=(50, 5), facecolor="w")
        fig.subplots_adjust(left=0.12, right=0.97, bottom=0.21, top=0.9, wspace=0.5)
        ax = fig.add_subplot(131)
        ax.plot(x, pdf)
        #ax.hist(cdf.reshape(-1, ), 30, density=True, histtype="stepfilled", alpha=0.9)
        ax.hist(data.reshape(-1, ), 30, density=True, histtype="stepfilled", alpha=0.5)
        plt.show()
        plt.close()
        """if test_stat[1] < 0.05:
            l += 1
            arr.append((i[0], test_stat[1]))
            fig_dir = "Singlefit<0.05"
            plt.plot(pdf)
            plt.axhline(y=target, color="r", xmax=1)
            if save_graph:
                j = 0
                while os.path.exists(f"figures/{fig_dir}/{i[0]}_{j}.png"):
                    j += 1
                plt.savefig(f"figures/{fig_dir}/{i[0]}_{j}.png", dpi=100, bbox_inches="tight", pad_inches=0.1)
            #plt.show()
            plt.close()
            
        else:
            k += 1
            fig_dir = "Singlefit>0.05"
            #print(i[0], target, test_stat[1])
            plt.plot(pdf)
            plt.axhline(y=target, color="r", xmax=1)
            if save_graph:
                j = 0
                while os.path.exists(f"figures/{fig_dir}/{i[0]}_{j}.png"):
                    j += 1
                plt.savefig(f"figures/{fig_dir}/{i[0]}_{j}.png", dpi=100, bbox_inches="tight", pad_inches=0.1)
            #plt.show()
            plt.close()"""
        
        
        # Apply gaussian mixture model to the data
        model, scores = gaussian_mixture(data, [target], components=2, scores_only=False)
        #if test_stat[1] > 0.05:
            #print("scores", scores)
            #print("ks", test_stat[1])
            #continue
        # Create list consisting of star name and its density for graph drawing
        scores.insert(0, target[0])
        scores.insert(0, np.log10(i[4].min()))
        scores.insert(0, np.log10(i[4].max()))
        scores.insert(0, np.log10(i[4].std()))
        scores.insert(0, np.log10(i[4].mean()))
        scores.insert(0, i[3])
        scores.insert(0, i[2])
        scores.insert(0, i[0])
        
        results.append(scores)
        
        # Draw best fit mixture
        if type(i[0]) != str:
            host = [f"{i[0]:.0f}", target]
        else:
            host = [i[0], target]
        
        if show_graph or save_graph:
            best_fit_mixture(model, data, host, results[densities.index(i)][9], fig_dir, show_graph, save_graph)
        if l > 1:
             break
    return l, k, arr
    df = pd.DataFrame(results, columns=["Host", "n_40pc_stars", "n_80pc_stars", "densities_mean", "densities_std", 
                                        "densities_max", "densities_min", "target_density", "gm_p_low", "gm_p_high",
                                        "gm_mean_low", "gm_mean_high", "gm_cov_low", "gm_cov_high", "gm_aic", "gm_bic"])
    df["class"] = df.apply(lambda row: labels(row), axis=1)
    
    df.to_csv(f"{classification_dir}/features_{file_name}.csv")